In [1]:
pip install nba_api

     |████████████████████████████████| 125 kB 1.6 MB/s eta 0:00:01
  Created wheel for nba-api: filename=nba_api-1.1.11-py3-none-any.whl size=251504 sha256=75a4cd75e6ca51574a71d4c7a43cbde6cecdf5943199d53201984ca5159f9ded
  Stored in directory: /Users/seanmcclellan/Library/Caches/pip/wheels/66/c2/3b/c87a243f9e5d2449e7f2c7bd65de4a6b5ce9a24b33978398a7
Successfully built nba-api
Note: you may need to restart the kernel to use updated packages.


In [23]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import requests
import json
import param
import numpy as np 
import panel as pn
pn.extension()
import pip 
import hvplot.pandas

In [24]:
from bs4 import BeautifulSoup
from nba_api.stats.endpoints import shotchartdetail

In [25]:
season_id = "2020-2021"
per_mode = "Totals"

In [26]:
player_info_url = "https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2020-21&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision=&Weight="

In [27]:
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [28]:
response = requests.get(url=player_info_url, headers=headers).json()

In [29]:
player_info = response['resultSets'][0]['rowSet']

In [30]:
columns_list = [
                "PLAYER_ID",
                "PLAYER_NAME",
                "NICKNAME",
                "TEAM_ID",
                "TEAM_ABBREVIATION",
                "AGE",
                "GP",
                "W",
                "L",
                "W_PCT",
                "MIN",
                "FGM",
                "FGA",
                "FG_PCT",
                "FG3M",
                "FG3A",
                "FG3_PCT",
                "FTM",
                "FTA",
                "FT_PCT",
                "OREB",
                "DREB",
                "REB",
                "AST",
                "TOV",
                "STL",
                "BLK",
                "BLKA",
                "PF",
                "PFD",
                "PTS",
                "PLUS_MINUS",
                "NBA_FANTASY_PTS",
                "DD2",
                "TD3",
                "GP_RANK",
                "W_RANK",
                "L_RANK",
                "W_PCT_RANK",
                "MIN_RANK",
                "FGM_RANK",
                "FGA_RANK",
                "FG_PCT_RANK",
                "FG3M_RANK",
                "FG3A_RANK",
                "FG3_PCT_RANK",
                "FTM_RANK",
                "FTA_RANK",
                "FT_PCT_RANK",
                "OREB_RANK",
                "DREB_RANK",
                "REB_RANK",
                "AST_RANK",
                "TOV_RANK",
                "STL_RANK",
                "BLK_RANK",
                "BLKA_RANK",
                "PF_RANK",
                "PFD_RANK",
                "PTS_RANK",
                "PLUS_MINUS_RANK",
                "NBA_FANTASY_PTS_RANK",
                "DD2_RANK",
                "TD3_RANK",
                "CFID",
                "CFPARAMS"
            ]

In [31]:
nba_df = pd.DataFrame(player_info, columns = columns_list)

In [32]:
response = shotchartdetail.ShotChartDetail(
    team_id=0,
    player_id=0,
    season_nullable='2020-21',
    season_type_all_star='Regular Season',
    context_measure_simple='FGA')

content = json.loads(response.get_json())

headers = {
        'Host': 'stats.nba.com',
        'Connection': 'keep-alive',
        'Accept': 'application/json, text/plain, */*',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
        'Referer': 'https://stats.nba.com/',
        "x-nba-stats-origin": "stats",
        "x-nba-stats-token": "true",
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
}

In [33]:
results = content['resultSets'][0]
headers = results['headers']
rows = results['rowSet']
shots = pd.DataFrame(rows)
shots.columns = headers

In [34]:
class NBA_ShotApp(param.Parameterized):
    
    team = param.ObjectSelector(default='Atlanta Hawks', objects=list(shots['TEAM_NAME']))
                                
    player = param.ObjectSelector(default='Bogdan Bogdanovic', objects=list(shots['PLAYER_NAME']))                            
    
    _players = {'Atlanta Hawks': ['Bogdan Bogdanovic', 'Clint Capela', 'John Collins', 'Kris Dunn', 'Bruno Fernando', 'Danilo Gallinari', 'Brandon Goodwin', 'Solomon Hill', 'Kevin Huerter', "De'Andre Hunter", 'Nathan Knight', 'Skylar Mays', 'Onyeka Okongwu', 'Cam Reddish', 'Rajon Rondo', 'Tony Snell', 'Lou Williams', 'Trae Young'],
                'Boston Celtics': ['Jaylen Brown', 'Carsen Edwards', 'Tacko Fall', 'Evan Fournier', 'Javonte Green', 'Robert Williams III', 'Luke Kornet', 'Romeo Langford', 'Aaron Nesmith', 'Semi Ojeleye', 'Jabari Parker', 'Payton Pritchard', 'Marcus Smart', 'Jayson Tatum', 'Jeff Teague', 'Daniel Theis', 'Tristan Thompson', 'Moritz Wagner', 'Kemba Walker', 'Tremont Waters', 'Grant Williams'],
                'Brooklyn Nets': ['LaMarcus Aldridge', 'Jarrett Allen', 'Bruce Brown', 'Chris Chiozza', 'Nic Claxton', 'Tyler Cook', 'Spencer Dinwiddie', 'Kevin Durant', 'Jeff Green', 'Blake Griffin', 'James Harden', 'Joe Harris', 'Kyrie Irving', 'Mike James', 'Tyler Johnson', 'Alize Johnson', 'DeAndre Jordan', 'Rodions Kurucs', 'Caris LeVert', 'Timothe Luwawu-Cabarrot', 'Norvel Pelle', 'Reggie Perry', 'Taurean Prince', 'Andre Roberson', 'Landry Shamet', 'Iman Shumpert', 'Noah Vonleh'],
                'Charlotte Hornets': ['LaMelo Ball', 'Bismack Biyombo', 'Miles Bridges', 'Nate Darling', "Devonte' Graham", 'Gordon Hayward', 'Vernon Carey Jr.', 'Cody Martin', 'Caleb Martin', 'Jalen McDaniels', 'Malik Monk', 'Nick Richards', 'Grant Riller', 'Terry Rozier', 'Brad Wanamaker', 'P.J. Washington', 'Cody Zeller'],
                'Chicago Bulls': ['Al-Farouq Aminu', 'Ryan Arcidiacono', 'Devon Dotson', 'Cristiano Felicio', 'Daniel Gafford', 'Javonte Green', 'Chandler Hutchison', 'Troy Brown Jr.', 'Otto Porter Jr.', 'Wendell Carter Jr.', 'Luke Kornet', 'Zach LaVine', 'Lauri Markkanen', 'Adam Mokoka', 'Tomas Satoransky', 'Garrett Temple', 'Daniel Theis', 'Denzel Valentine', 'Nikola Vucevic', 'Coby White', 'Patrick Williams', 'Thaddeus Young'],
                'Cleveland Cavaliers': ['Jarrett Allen', 'Marques Bolden', 'Quinn Cook', 'Matthew Dellavedova', 'Damyean Dotson', 'Andre Drummond', 'Dante Exum', 'Yogi Ferrell', 'Darius Garland', 'Isaiah Hartenstein', 'Larry Nance Jr.', 'Mfiondu Kabengele', 'Kevin Love', 'Thon Maker', 'Jeremiah Martin', 'JaVale McGee', 'Isaac Okoro', 'Cedi Osman', 'Taurean Prince', 'Collin Sexton', 'Lamar Stevens', 'Brodric Thomas', 'Anderson Varejao', 'Dean Wade', 'Dylan Windler'],
                'Dallas Mavericks': ['Tyler Bey', 'Jalen Brunson', 'Trey Burke', 'Willie Cauley-Stein', 'Luka Doncic', 'Dorian Finney-Smith', 'Josh Green', 'Nate Hinton', 'Wes Iwundu', 'James Johnson', 'Tim Hardaway Jr.', 'Maxi Kleber', 'Boban Marjanovic', 'Nicolo Melli', 'Kristaps Porzingis', 'Dwight Powell', 'JJ Redick', 'Josh Richardson', 'Tyrell Terry'],
                'Denver Nuggets': ['Will Barton', 'Bol Bol', 'Facundo Campazzo', 'Vlatko Cancar', 'P.J. Dozier', 'Aaron Gordon', 'JaMychal Green', 'R.J. Hampton', 'Gary Harris', 'Shaquille Harrison', 'Isaiah Hartenstein', 'Markus Howard', 'Nikola Jokic', 'Michael Porter Jr.', 'JaVale McGee', 'Paul Millsap', 'Monte Morris', 'Jamal Murray', 'Zeke Nnaji', 'Austin Rivers', 'Greg Whittington'],
                'Detroit Pistons': ['Saddiq Bey', 'Tyler Cook', 'Hamidou Diallo', 'Sekou Doumbouya', 'Wayne Ellington', 'Jerami Grant', 'Blake Griffin', 'Killian Hayes', 'Josh Jackson', 'Frank Jackson', 'Cory Joseph', 'Dennis Smith Jr.', 'Saben Lee', 'Rodney McGruder', 'Svi Mykhailiuk', 'Jahlil Okafor', 'Mason Plumlee', 'Derrick Rose', 'Deividas Sirvydis', 'Isaiah Stewart', 'Delon Wright'],
                'Golden State Warriors': ['Kent Bazemore', 'Jordan Bell', 'Marquese Chriss', 'Stephen Curry', 'Draymond Green', 'Gary Payton II', 'Kelly Oubre Jr.', 'Damion Lee', 'Kevon Looney', 'Nico Mannion', 'Mychal Mulder', 'Eric Paschall', 'Jordan Poole', 'Alen Smailagic', 'Juan Toscano-Anderson', 'Brad Wanamaker', 'Andrew Wiggins', 'James Wiseman'],
                'Houston Rockets': ['D.J. Augustin', 'Avery Bradley', 'Armoni Brooks', 'Sterling Brown', 'Bruno Caboclo', 'DeMarcus Cousins', 'Eric Gordon', 'James Harden', 'DaQuan Jeffries', 'Mason Jones', 'Danuel House Jr.', 'Kevin Porter Jr.', 'Kenyon Martin Jr.', 'Rodions Kurucs', 'Anthony Lamb', 'Ben McLemore', 'David Nwaba', 'Victor Oladipo', 'Cameron Oliver', 'Kelly Olynyk', 'Justin Patton', 'Cameron Reynolds', 'Ray Spalding', "Jae'Sean Tate", 'Khyri Thomas', 'Brodric Thomas', 'P.J. Tucker', 'John Wall', 'D.J. Wilson', 'Christian Wood'],
                'Indiana Pacers': ['Goga Bitadze', 'Amida Brimah', 'Oshae Brissett', 'Malcolm Brogdon', 'Justin Holiday', 'Aaron Holiday', 'Brian Bowen II', 'Jeremy Lamb', 'Caris LeVert', 'Jalen Lecque', 'Kelan Martin', 'T.J. McConnell', 'Doug McDermott', 'Victor Oladipo', 'Domantas Sabonis', 'JaKarr Sampson', 'Cassius Stanley', 'Edmond Sumner', 'Myles Turner', 'T.J. Warren'],
                'LA Clippers': ['Nicolas Batum', 'Patrick Beverley', 'Amir Coffey', 'DeMarcus Cousins', 'Yogi Ferrell', 'Malik Fitts', 'Paul George', 'Serge Ibaka', 'Reggie Jackson', 'Mfiondu Kabengele', 'Luke Kennard', 'Kawhi Leonard', 'Terance Mann', 'Daniel Oturu', 'Patrick Patterson', 'Rajon Rondo', 'Jay Scrubb', 'Marcus Morris Sr.', 'Lou Williams', 'Ivica Zubac'],
                'Los Angeles Lakers': ['Kostas Antetokounmpo', 'Devontae Cacok', 'Kentavious Caldwell-Pope', 'Alex Caruso', 'Quinn Cook', 'Anthony Davis', 'Andre Drummond', 'Jared Dudley', 'Marc Gasol', 'Montrezl Harrell', 'Talen Horton-Tucker', 'LeBron James', 'Damian Jones', 'Kyle Kuzma', 'Wesley Matthews', 'Alfonzo McKinnie', 'Ben McLemore', 'Markieff Morris', 'Dennis Schroder'],
                'Memphis Grizzlies': ['Grayson Allen', 'Kyle Anderson', 'Desmond Bane', 'Dillon Brooks', 'Brandon Clarke', 'Gorgui Dieng', 'Tim Frazier', 'Tyus Jones', 'Jaren Jackson Jr.', 'John Konchar', 'Sean McDermott', "De'Anthony Melton", 'Ja Morant', 'Jontay Porter', 'Killian Tillie', 'Xavier Tillman', 'Jonas Valanciunas', 'Justise Winslow'],
                'Miami Heat': ['Precious Achiuwa', 'Bam Adebayo', 'Trevor Ariza', 'Nemanja Bjelica', 'Avery Bradley', 'Jimmy Butler', 'Dewayne Dedmon', 'Goran Dragic', 'Maurice Harkless', 'Udonis Haslem', 'Tyler Herro', 'Andre Iguodala', 'Meyers Leonard', 'Kendrick Nunn', 'KZ Okpala', 'Victor Oladipo', 'Kelly Olynyk', 'Duncan Robinson', 'Chris Silva', 'Max Strus', 'Gabe Vincent'],
                'Milwaukee Bucks': ['Jaylen Adams', 'Giannis Antetokounmpo', 'Thanasis Antetokounmpo', 'D.J. Augustin', 'Elijah Bryant', 'Pat Connaughton', 'Torrey Craig', 'Donte DiVincenzo', 'Mamadi Diakite', 'Bryn Forbes', 'Jrue Holiday', 'Justin Jackson', 'Rodions Kurucs', 'Brook Lopez', 'Sam Merrill', 'Khris Middleton', 'Jordan Nwora', 'Bobby Portis', 'Jeff Teague', 'Axel Toupane', 'P.J. Tucker', 'D.J. Wilson'],
                'Minnesota Timberwolves': ['Malik Beasley', 'Jarrett Culver', 'Ed Davis', 'Anthony Edwards', 'Juancho Hernangomez', 'Jake Layman', 'Jaden McDaniels', 'Jordan McLaughlin', 'Jaylen Nowell', 'Josh Okogie', 'Naz Reid', 'Ricky Rubio', "D'Angelo Russell", 'Karl-Anthony Towns', 'Jarred Vanderbilt'],
                'New Orleans Pelicans': ['Steven Adams', 'Nickeil Alexander-Walker', 'Lonzo Ball', 'Eric Bledsoe', 'Wenyen Gabriel', 'Josh Hart', 'Jaxson Hayes', 'Willy Hernangomez', 'Brandon Ingram', 'Wes Iwundu', 'James Johnson', 'Kira Lewis Jr.', 'Didi Louzada', 'Will Magnay', 'Naji Marshall', 'Nicolo Melli', 'James Nunnally', 'JJ Redick', 'Isaiah Thomas', 'Sindarius Thornwell', 'Zion Williamson'],
                'New York Knicks': ['RJ Barrett', 'Ignas Brazdeikis', 'Reggie Bullock', 'Alec Burks', 'Taj Gibson', 'Jared Harper', 'Kevin Knox II', 'Dennis Smith Jr.', 'Nerlens Noel', 'Frank Ntilikina', 'Elfrid Payton', 'Norvel Pelle', 'Theo Pinson', 'Immanuel Quickley', 'Julius Randle', 'Austin Rivers', 'Mitchell Robinson', 'Derrick Rose', 'Obi Toppin'],
                'Oklahoma City Thunder': ['Darius Bazley', 'Tony Bradley', 'Moses Brown', 'Gabriel Deck', 'Hamidou Diallo', 'Luguentz Dort', 'Shai Gilgeous-Alexander', 'Josh Hall', 'George Hill', 'Jaylen Hoard', 'Al Horford', 'Justin Jackson', 'Ty Jerome', 'Charlie Brown Jr.', 'Theo Maledon', 'Darius Miller', 'Mike Muscala', 'Svi Mykhailiuk', 'Aleksej Pokusevski', 'Justin Robinson', 'Isaiah Roby', 'Kenrich Williams'],
                'Orlando Magic': ['Al-Farouq Aminu', 'Cole Anthony', 'Dwayne Bacon', 'Mo Bamba', 'Khem Birch', 'Jordan Bone', 'Ignas Brazdeikis', 'Devin Cannady', 'Michael Carter-Williams', 'Gary Clark', 'Evan Fournier', 'Robert Franks', 'Markelle Fultz', 'Aaron Gordon', 'Donta Hall', 'R.J. Hampton', 'Gary Harris', 'Frank Mason III', 'James Ennis III', 'Otto Porter Jr.', 'Wendell Carter Jr.', 'Karim Mane', 'Chuma Okeke', 'Chasson Randle', 'Terrence Ross', 'Sindarius Thornwell', 'Nikola Vucevic', 'Moritz Wagner'],
                'Philadelphia 76ers': ['Tony Bradley', 'Ignas Brazdeikis', 'Gary Clark', 'Seth Curry', 'Joel Embiid', 'Terrance Ferguson', 'Danny Green', 'Tobias Harris', 'George Hill', 'Dwight Howard', 'Isaiah Joe', 'Mason Jones', 'Furkan Korkmaz', 'Dakota Mathias', 'Tyrese Maxey', 'Shake Milton', 'Vincent Poirier', 'Paul Reed', 'Mike Scott', 'Ben Simmons', 'Matisse Thybulle', 'Anthony Tolliver', 'Rayjon Tucker'],
                'Phoenix Suns': ['Ty-Shon Alexander', 'Deandre Ayton', 'Devin Booker', 'Mikal Bridges', 'Jevon Carter', 'Torrey Craig', 'Jae Crowder', 'Langston Galloway', 'Cameron Johnson', 'Damian Jones', 'Frank Kaminsky', "E'Twaun Moore", 'Abdel Nader', 'Chris Paul', 'Cameron Payne', 'Dario Saric', 'Jalen Smith'],
                'Portland Trail Blazers': ['Carmelo Anthony', 'Keljin Blevins', 'Robert Covington', 'CJ Elleby', 'Enes Freedom', 'Rondae Hollis-Jefferson', 'Rodney Hood', 'Harry Giles III', 'Gary Trent Jr.', 'Derrick Jones Jr.', 'T.J. Leaf', 'Damian Lillard', 'Nassir Little', 'CJ McCollum', 'Jusuf Nurkic', 'Norman Powell', 'Anfernee Simons'],
                'Sacramento Kings': ['Harrison Barnes', 'Nemanja Bjelica', 'Terence Davis', "De'Aaron Fox", 'Kyle Guy', 'Tyrese Haliburton', 'Maurice Harkless', 'Buddy Hield', 'Richaun Holmes', 'Robert Woodard II', 'Marvin Bagley III', 'Glenn Robinson III', 'Justin James', 'DaQuan Jeffries', 'Damian Jones', 'Cory Joseph', 'Louis King', 'Chimezie Metu', 'Jabari Parker', 'Norvel Pelle', "Jahmi'us Ramsey", 'Chris Silva', 'Hassan Whiteside', 'Delon Wright'],
                'San Antonio Spurs': ['LaMarcus Aldridge', 'Keita Bates-Diop', 'DeMar DeRozan', 'Gorgui Dieng', 'Drew Eubanks', 'Rudy Gay', 'Lonnie Walker IV', 'Keldon Johnson', 'Tre Jones', 'Trey Lyles', 'Patty Mills', 'Dejounte Murray', 'Jakob Poeltl', 'Cameron Reynolds', 'Luka Samanic', 'Devin Vassell', 'Quinndary Weatherspoon', 'Derrick White'],
                'Toronto Raptors': ['OG Anunoby', 'Aron Baynes', "DeAndre' Bembry", 'Khem Birch', 'Chris Boucher', 'Terence Davis', 'Henry Ellenson', 'Malachi Flynn', 'Freddie Gillespie', 'Jalen Harris', 'Rodney Hood', 'Stanley Johnson', 'Gary Trent Jr.', 'Alex Len', 'Kyle Lowry', 'Patrick McCaw', 'Norman Powell', 'Pascal Siakam', 'Matt Thomas', 'Fred VanVleet', 'Yuta Watanabe', 'Paul Watson'],
                'Utah Jazz': ['Udoka Azubuike', 'Bojan Bogdanovic', 'Jarrell Brantley', 'Jordan Clarkson', 'Mike Conley', 'Derrick Favors', 'Trent Forrest', 'Rudy Gobert', 'Shaquille Harrison', 'Elijah Hughes', 'Ersan Ilyasova', 'Joe Ingles', 'Donovan Mitchell', 'Juwan Morgan', 'Georges Niang', "Royce O'Neale", 'Miye Oni', 'Matt Thomas'],
                'Washington Wizards': ['Deni Avdija', 'Bradley Beal', 'Jordan Bell', 'Davis Bertans', 'Isaac Bonga', 'Thomas Bryant', 'Daniel Gafford', 'Anthony Gill', 'Rui Hachimura', 'Chandler Hutchison', 'Troy Brown Jr.', 'Alex Len', 'Robin Lopez', 'Garrison Mathews', 'Raul Neto', 'Anzejs Pasecniks', 'Jerome Robinson', 'Ish Smith', 'Moritz Wagner', 'Russell Westbrook', 'Cassius Winston']}
    
    @param.depends('team', watch=True)
    def _update_players(self):
        players = self._players[self.team]
        self.param['player'].objects = players
        self.player = players[0]

    @param.depends('team',watch=True)
    def view_team(self):
        team_shot_data = shots.loc[shots['TEAM_NAME'] == self.team]
        layout_one = team_shot_data.hvplot(x="LOC_X", y="LOC_Y",by="EVENT_TYPE",legend="left",alpha=0.4,width=425,dynamic=True,kind="scatter", hover_cols=['SHOT_DISTANCE','SHOT_TYPE','ACTION_TYPE','GAME_DATE','PLAYER_NAME']).opts(title="Team Shots",fontsize={'title': 16, 'labels': 0, 'xticks': 0, 'yticks': 0})
        return layout_one
        
    @param.depends('player')
    def view_player(self):
        player_shot_data = shots.loc[shots['PLAYER_NAME'] == self.player]
        layout_two = player_shot_data.hvplot(x="LOC_X", y="LOC_Y",by="EVENT_TYPE",width=275,dynamic=True,legend=False,alpha=0.4,kind="scatter",hover_cols=['SHOT_DISTANCE','SHOT_TYPE','ACTION_TYPE','GAME_DATE','PLAYER_NAME']).opts(title="Player Shots",fontsize={'title': 16, 'labels': 0, 'xticks': 0, 'yticks': 0})
        return layout_two
    
    @param.depends('player')
    def view_stats(self):
        player_stat_data = nba_df.loc[nba_df['PLAYER_NAME'] == self.player]
        layout_three = player_stat_data.hvplot.table(columns=['AGE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS'], width= 1000, sortable=True, selectable=True).opts(title="Player Stats",fontsize={'title': 16})
        return layout_three
         
app = NBA_ShotApp(name='<h2>Shot Location in 2020-2021 Season by Team and Player<h2>')

shot_dashboard = pn.Column(
    pn.Row(
        app.param, app.view_team, app.view_player),
    pn.Row(app.view_stats))

In [35]:
shot_dashboard.servable()

Column
    [0] Row
        [0] Column(margin=5, name='<h2>Shot Location i..., width=300)
            [0] StaticText(value='<b><h2>Shot L...)
            [1] Select(name='Team', options=OrderedDict([('Brooklyn Ne...]), value='Atlanta Hawks')
            [2] Select(name='Player', options=OrderedDict([('Kyrie Irvin...]), value='Bogdan Bogdanovic')
        [1] ParamMethod(method)
        [2] ParamMethod(method)
    [1] Row
        [0] ParamMethod(method)